In [1]:
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
import JLD2
using Statistics
using MLJBase
using Printf
using BSON: @load # for load weights
using BSON: @save # for load weights
using Plots
using Zygote
using Distributions

In [4]:
train_data = JLD2.load_object("data/train_data.jld2");
test_data = JLD2.load_object("data/test_data.jld2");
x_train = JLD2.load_object("data/x_train.jld2");
y_train = JLD2.load_object("data/y_train.jld2");
x_test  = JLD2.load_object("data/x_test.jld2");
y_test  = JLD2.load_object("data/y_test.jld2");

In [13]:
epochs = 200
N = 10
w0 = 0.8
c1 = 1.6
c2 = 1.8


1.8

In [6]:
accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)
loss(model) = (x,y)->logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

In [ ]:
save_resultTest = zeros(epochs);
save_resultTrain = zeros(epochs);
rng =Random.default_rng()

In [7]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))#define the model 
θ, re = Flux.destructure(model)


Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 3.523 KiB.

In [34]:
weights = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:N];
vec = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:N];

In [36]:
for i in 1:N
    for j in 1:size(θ)[1]
        weights[i][j] = rand(Normal(θ[i], 0.05))
        vec[i] = rand(size(θ)[1])
    end
    
end
p_best = copy(weights);
g_best = sort(weights, lt=(x,y)->isless(loss(train_data, re(x)), loss(train_data, re(y))))[1];


In [ ]:

for i in 1:epochs
    for j in 1:N
        vec[j] = w0 *vec[j] + c1 *(1.0 - rand()) * (p_best[j] - vec[j]) + c2 * (1.0 - rand()) * (g_best - vec[j])
        weights[j] = vec[j]+weights[j]
        if loss(train_data, re(weights[j])) < loss(train_data, re(p_best[j]))
            p_best[i] = weights[i]
        end
    end
    g_best = sort(weights, lt=(x,y)->isless(loss(train_data, re(x)), loss(train_data, re(y))))[1];
    printf("Epoch %d: %f\n", i, loss(train_data, re(g_best)));
end

In [107]:

for i in 1:10
    if loss(train_data, re(weights[i])) < loss(train_data, re(p_best[i]))
        p_best[i] = weights[i]
    end
        
end


In [55]:
model = re(weights[1])

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 6.797 KiB.

In [120]:
test = copy(weights[10])
ptest = copy(p_best[10])

838-element Vector{Float64}:
 0.14829884511400043
 0.10556157852396715
 0.21230379488839088
 0.2185256340858201
 0.15001595168989
 0.1792533066592839
 0.16119050368450855
 0.1332225044449725
 0.19696960690961318
 0.062540763167726
 0.1351317873372226
 0.15159808171080752
 0.1861870703251785
 ⋮
 0.11864318218440921
 0.2902619616303841
 0.13955342265165877
 0.21071064677356632
 0.10298687173721453
 0.1435496963419464
 0.11055660439794684
 0.2021934359674416
 0.21686852473667856
 0.131384412611903
 0.2028528613488662
 0.09102736949122608

In [96]:
1.0 - rand()

0.45947884197301814

In [111]:
curV = 1.0

1.0

In [121]:
ptest - test

838-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [147]:
newV =   newV .+ 2 *(1.0 - rand())*(ptest - test) + 2*(1.0 - rand())*(g_best - test)
test = test .* newV

838-element Vector{Float64}:
      -2.127635045314713e-5
       1.5565729371214304e-14
     -Inf
      -5.283256130289876e294
      -2.290520043703234e6
      -1.448474683479714e31
      -4.424937941561609e69
      -5.1975065551027104e-8
     -Inf
       2.4485111240880432e-32
      -1.2117191600327958e171
    -945.1784466180011
      -0.829146007799524
       ⋮
      -2.0557804137621827
     -Inf
      -0.002073792980130473
  -13077.961919278285
 -256141.4005424382
      -3.6978701260244558e6
      -0.004965873605598218
      -1.200383167717702e304
      -1.9591499884150424e206
      -8.726046094429081e27
     -Inf
       0.0007051019418867873

In [2]:
v = [-1.0,-1.5]
x = [3.0,4.0]
p = [2.5,3.6]
g = [2.3,3.4]

2-element Vector{Float64}:
 2.3
 3.4

In [3]:
v = 0.7*v + 1.4*0.5*(p-x) + 1.4*0.6*(g-x)
x = x+v

2-element Vector{Float64}:
 1.362
 2.1660000000000004